In [39]:
import re
import matplotlib.pyplot as plt
import time
import numpy as np
import datetime
%matplotlib inline

In [25]:
abstracts = '/home/ctroupin/abstract_list'

In [26]:
year, month, day = [], [], []

In [44]:
f = open(abstracts, 'r')
dates = np.array([])
for lines in f:
    match = re.search(r'(\d{4})\-(\d{2})\-(\d{2})', lines)
    if match:
        year.append(match.group(1))
        month.append(match.group(2))
        day.append(match.group(3))
        dates = np.append(dates, datetime.datetime(int(year[-1]), int(month[-1]), int(day[-1]), 0, 0))
f.close()

In [45]:
type(dates)

numpy.ndarray

In [47]:
time.mktime(dates[0].timetuple())

1442959200.0